In [2]:
import pandas as pd
import os
import numpy as np
#import matplotlib.pyplot as plt
#import seaborn as sns
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import cohen_kappa_score
from scipy.stats import mode
from sklearn.feature_selection import SelectFromModel
from sklearn.model_selection import train_test_split
#import xgboost as xgb
#from xgboost import XGBClassifier
#from xgboost import plot_importance
#from matplotlib import pyplot

In [3]:
def _log(str):
    os.system(f'echo \"{str}\"')
    print(str)

In [4]:
INPUT_ROOT = '../input/data-science-bowl-2019'
JOIN_KEY = ['installation_id', 'game_session', 'title']
TARGET = 'accuracy_group'
CATEGORICAL_FEATURES = {'event_id', 'timestamp', 'event_data', 'event_code', 'title', 'type', 'world'}

In [5]:
def _init():
    # Characters such as empty strings '' or numpy.inf are considered NA values
    pd.set_option('use_inf_as_na', True)
    pd.set_option('display.max_columns', 500)
    
    
_init()

In [6]:
# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

for dirname, _, filenames in os.walk(INPUT_ROOT):
    for filename in filenames:
        _log(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

../input/data-science-bowl-2019\test.csv
../input/data-science-bowl-2019\test.csv.zip
../input/data-science-bowl-2019\train.csv
../input/data-science-bowl-2019\train.csv.zip
../input/data-science-bowl-2019\train_labels.csv
../input/data-science-bowl-2019\train_labels.csv.zip


In [7]:
%%time
train_raw = pd.read_csv(f'{INPUT_ROOT}/train.csv')
train_labels = pd.read_csv(f'{INPUT_ROOT}/train_labels.csv', usecols=JOIN_KEY + [TARGET])
test = pd.read_csv(f'{INPUT_ROOT}/test.csv')
train_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11341042 entries, 0 to 11341041
Data columns (total 11 columns):
event_id           object
game_session       object
timestamp          object
event_data         object
installation_id    object
event_count        int64
event_code         int64
game_time          int64
title              object
type               object
world              object
dtypes: int64(3), object(8)
memory usage: 951.8+ MB
Wall time: 52.2 s


In [8]:
vs = sorted(train_raw['title'].unique())
_log(f'{len(vs)} train_raw titles={vs}')

44 train_raw titles=['12 Monkeys', 'Air Show', 'All Star Sorting', 'Balancing Act', 'Bird Measurer (Assessment)', 'Bottle Filler (Activity)', 'Bubble Bath', 'Bug Measurer (Activity)', 'Cart Balancer (Assessment)', 'Cauldron Filler (Assessment)', 'Chest Sorter (Assessment)', 'Chicken Balancer (Activity)', 'Chow Time', 'Costume Box', 'Crystal Caves - Level 1', 'Crystal Caves - Level 2', 'Crystal Caves - Level 3', 'Crystals Rule', 'Dino Dive', 'Dino Drink', 'Egg Dropper (Activity)', 'Fireworks (Activity)', 'Flower Waterer (Activity)', 'Happy Camel', 'Heavy, Heavier, Heaviest', 'Honey Cake', 'Leaf Leader', 'Lifting Heavy Things', 'Magma Peak - Level 1', 'Magma Peak - Level 2', 'Mushroom Sorter (Assessment)', 'Ordering Spheres', 'Pan Balance', "Pirate's Tale", 'Rulers', 'Sandcastle Builder (Activity)', 'Scrub-A-Dub', 'Slop Problem', 'Treasure Map', 'Tree Top City - Level 1', 'Tree Top City - Level 2', 'Tree Top City - Level 3', 'Watering Hole (Activity)', 'Welcome to Lost Lagoon!']


In [9]:
vs = sorted(test['title'].unique())
_log(f'{len(vs)} test titles={vs}')

44 test titles=['12 Monkeys', 'Air Show', 'All Star Sorting', 'Balancing Act', 'Bird Measurer (Assessment)', 'Bottle Filler (Activity)', 'Bubble Bath', 'Bug Measurer (Activity)', 'Cart Balancer (Assessment)', 'Cauldron Filler (Assessment)', 'Chest Sorter (Assessment)', 'Chicken Balancer (Activity)', 'Chow Time', 'Costume Box', 'Crystal Caves - Level 1', 'Crystal Caves - Level 2', 'Crystal Caves - Level 3', 'Crystals Rule', 'Dino Dive', 'Dino Drink', 'Egg Dropper (Activity)', 'Fireworks (Activity)', 'Flower Waterer (Activity)', 'Happy Camel', 'Heavy, Heavier, Heaviest', 'Honey Cake', 'Leaf Leader', 'Lifting Heavy Things', 'Magma Peak - Level 1', 'Magma Peak - Level 2', 'Mushroom Sorter (Assessment)', 'Ordering Spheres', 'Pan Balance', "Pirate's Tale", 'Rulers', 'Sandcastle Builder (Activity)', 'Scrub-A-Dub', 'Slop Problem', 'Treasure Map', 'Tree Top City - Level 1', 'Tree Top City - Level 2', 'Tree Top City - Level 3', 'Watering Hole (Activity)', 'Welcome to Lost Lagoon!']


In [10]:
vs = sorted(train_labels['title'].unique())
_log(f'{len(vs)} train_labels titles={vs}')

5 train_labels titles=['Bird Measurer (Assessment)', 'Cart Balancer (Assessment)', 'Cauldron Filler (Assessment)', 'Chest Sorter (Assessment)', 'Mushroom Sorter (Assessment)']


In [11]:
train_labels.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17690 entries, 0 to 17689
Data columns (total 4 columns):
game_session       17690 non-null object
installation_id    17690 non-null object
title              17690 non-null object
accuracy_group     17690 non-null int64
dtypes: int64(1), object(3)
memory usage: 552.9+ KB


In [12]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1156414 entries, 0 to 1156413
Data columns (total 11 columns):
event_id           1156414 non-null object
game_session       1156414 non-null object
timestamp          1156414 non-null object
event_data         1156414 non-null object
installation_id    1156414 non-null object
event_count        1156414 non-null int64
event_code         1156414 non-null int64
game_time          1156414 non-null int64
title              1156414 non-null object
type               1156414 non-null object
world              1156414 non-null object
dtypes: int64(3), object(8)
memory usage: 97.1+ MB


In [13]:
%%time
train = pd.merge(train_raw, train_labels, on=JOIN_KEY, how='inner')

Wall time: 2.4 s


In [14]:
def _drop_unused_columns(df):
    cols = ['game_session', 'type']
    return df.drop(columns=cols)

    
#train = _drop_unused_columns(train)
#test = _drop_unused_columns(test)

In [15]:
%%time
def _set_string_type(df, cols):
    df[cols] = df[cols].astype(str)
    return df


#cols = list(CATEGORICAL_FEATURES)
#train = _set_string_type(train, cols=cols + [TARGET])
#test = _set_string_type(test, cols=cols)

Wall time: 0 ns


In [16]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 865447 entries, 0 to 865446
Data columns (total 12 columns):
event_id           865447 non-null object
game_session       865447 non-null object
timestamp          865447 non-null object
event_data         865447 non-null object
installation_id    865447 non-null object
event_count        865447 non-null int64
event_code         865447 non-null int64
game_time          865447 non-null int64
title              865447 non-null object
type               865447 non-null object
world              865447 non-null object
accuracy_group     865447 non-null int64
dtypes: int64(4), object(8)
memory usage: 85.8+ MB


In [17]:
def _log_smoothing(df, cols):
    for col in cols:
        df[col] = np.log(df[col] + 1)
    return df


#cols = ['event_count', 'game_time']
#train = _log_smoothing(train, cols)
#test = _log_smoothing(test, cols)

In [18]:
train.head()

,event_id,game_session,timestamp,event_data,installation_id,event_count,event_code,game_time,title,type,world,accuracy_group
0,3bfd1a65,901acc108f55a5a1,2019-08-06T05:22:01.344Z,"{""version"":""1.0"",""event_count"":1,""game_time"":0...",0006a69f,1,2000,0,Mushroom Sorter (Assessment),Assessment,TREETOPCITY,3
1,db02c830,901acc108f55a5a1,2019-08-06T05:22:01.400Z,"{""event_count"":2,""game_time"":37,""event_code"":2...",0006a69f,2,2025,37,Mushroom Sorter (Assessment),Assessment,TREETOPCITY,3
2,a1e4395d,901acc108f55a5a1,2019-08-06T05:22:01.403Z,"{""description"":""Pull three mushrooms out of th...",0006a69f,3,3010,37,Mushroom Sorter (Assessment),Assessment,TREETOPCITY,3
3,a52b92d5,901acc108f55a5a1,2019-08-06T05:22:05.242Z,"{""description"":""Pull three mushrooms out of th...",0006a69f,4,3110,3901,Mushroom Sorter (Assessment),Assessment,TREETOPCITY,3
4,a1e4395d,901acc108f55a5a1,2019-08-06T05:22:05.244Z,"{""description"":""To pick a mushroom, pull it ou...",0006a69f,5,3010,3901,Mushroom Sorter (Assessment),Assessment,TREETOPCITY,3


In [19]:
train.describe(include='all')

,event_id,game_session,timestamp,event_data,installation_id,event_count,event_code,game_time,title,type,world,accuracy_group
count,865447,865447,865447,865447,865447,865447.000000,865447.000000,8.654470e+05,865447,865447,865447,865447.000000
unique,97,17690,842415,805123,3614,NaN,NaN,NaN,5,1,3,NaN
top,a8efe47b,a229f001486f628c,2019-08-05T20:25:40.118Z,"{""version"":""1.0"",""event_count"":1,""game_time"":0...",08987c08,NaN,NaN,NaN,Bird Measurer (Assessment),Assessment,TREETOPCITY,NaN
freq,45693,1274,4,17690,7395,NaN,NaN,NaN,184767,865447,365307,NaN
mean,NaN,NaN,NaN,NaN,NaN,37.914465,3593.477654,6.218941e+04,NaN,NaN,NaN,1.533335
std,NaN,NaN,NaN,NaN,NaN,50.163982,655.223229,9.601085e+05,NaN,NaN,NaN,1.240931
min,NaN,NaN,NaN,NaN,NaN,1.000000,2000.000000,0.000000e+00,NaN,NaN,NaN,0.000000
25%,NaN,NaN,NaN,NaN,NaN,13.000000,3110.000000,1.008300e+04,NaN,NaN,NaN,0.000000
50%,NaN,NaN,NaN,NaN,NaN,26.000000,4025.000000,2.284900e+04,NaN,NaN,NaN,1.000000
75%,NaN,NaN,NaN,NaN,NaN,46.000000,4035.000000,4.479300e+04,NaN,NaN,NaN,3.000000


In [20]:
test.head()

,event_id,game_session,timestamp,event_data,installation_id,event_count,event_code,game_time,title,type,world
0,27253bdc,0ea9ecc81a565215,2019-09-10T16:50:24.910Z,"{""event_code"": 2000, ""event_count"": 1}",00abaee7,1,2000,0,Welcome to Lost Lagoon!,Clip,NONE
1,27253bdc,c1ea43d8b8261d27,2019-09-10T16:50:55.503Z,"{""event_code"": 2000, ""event_count"": 1}",00abaee7,1,2000,0,Magma Peak - Level 1,Clip,MAGMAPEAK
2,27253bdc,7ed86c6b72e725e2,2019-09-10T16:51:51.805Z,"{""event_code"": 2000, ""event_count"": 1}",00abaee7,1,2000,0,Magma Peak - Level 2,Clip,MAGMAPEAK
3,27253bdc,7e516ace50e7fe67,2019-09-10T16:53:12.825Z,"{""event_code"": 2000, ""event_count"": 1}",00abaee7,1,2000,0,Crystal Caves - Level 1,Clip,CRYSTALCAVES
4,7d093bf9,a022c3f60ba547e7,2019-09-10T16:54:12.115Z,"{""version"":""1.0"",""round"":0,""event_count"":1,""ga...",00abaee7,1,2000,0,Chow Time,Game,CRYSTALCAVES


In [21]:
test.describe(include='all')

,event_id,game_session,timestamp,event_data,installation_id,event_count,event_code,game_time,title,type,world
count,1156414,1156414,1156414,1156414,1156414,1.156414e+06,1.156414e+06,1.156414e+06,1156414,1156414,1156414
unique,365,28445,1103166,1120854,1000,NaN,NaN,NaN,44,4,4
top,bb3e370b,b246933e8ac6a4aa,2019-09-13T18:38:38.848Z,"{""event_code"": 2000, ""event_count"": 1}",7b728c89,NaN,NaN,NaN,Bottle Filler (Activity),Game,MAGMAPEAK
freq,28667,4456,9,15253,21237,NaN,NaN,NaN,112223,572260,511291
mean,NaN,NaN,NaN,NaN,NaN,1.107075e+02,3.514353e+03,1.869646e+05,NaN,NaN,NaN
std,NaN,NaN,NaN,NaN,NaN,2.193408e+02,6.802671e+02,1.670162e+06,NaN,NaN,NaN
min,NaN,NaN,NaN,NaN,NaN,1.000000e+00,2.000000e+03,0.000000e+00,NaN,NaN,NaN
25%,NaN,NaN,NaN,NaN,NaN,2.500000e+01,3.021000e+03,3.056125e+04,NaN,NaN,NaN
50%,NaN,NaN,NaN,NaN,NaN,6.000000e+01,4.020000e+03,7.102800e+04,NaN,NaN,NaN
75%,NaN,NaN,NaN,NaN,NaN,1.240000e+02,4.035000e+03,1.476910e+05,NaN,NaN,NaN
